## Imports and Loading Data 

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score,confusion_matrix, classification_report, roc_curve, auc 
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Load fraud detection dataset
data = pd.read_csv('../Raw Data /PS_20174392719_1491204439457_log.csv')
data_ = pd.DataFrame(data)
data_.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


In [14]:
data.tail(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362610,742,TRANSFER,"$63,416.99",C778071008,NaN,NaN,C1812552860,NaN,NaN,1,0
6362611,742,CASH_OUT,"$63,416.99",C994950684,NaN,NaN,C1662241365,NaN,NaN,1,0
6362612,743,TRANSFER,"$1,258,818.82",C1531301470,NaN,NaN,C1470998563,NaN,NaN,1,0
6362613,743,CASH_OUT,"$1,258,818.82",C1436118706,NaN,NaN,C1240760502,NaN,NaN,1,0
6362614,743,TRANSFER,"$339,682.13",C2013999242,NaN,NaN,C1850423904,NaN,NaN,1,0
6362615,743,CASH_OUT,"$339,682.13",C786484425,NaN,NaN,C776919290,NaN,NaN,1,0
6362616,743,TRANSFER,"$6,311,409.28",C1529008245,NaN,NaN,C1881841831,NaN,NaN,1,0
6362617,743,CASH_OUT,"$6,311,409.28",C1162922333,NaN,NaN,C1365125890,NaN,NaN,1,0
6362618,743,TRANSFER,"$850,002.52",C1685995037,NaN,NaN,C2080388513,NaN,NaN,1,0
6362619,743,CASH_OUT,"$850,002.52",C1280323807,NaN,NaN,C873221189,NaN,NaN,1,0


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2770409 entries, 2 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   step            int64 
 1   type            object
 2   amount          object
 3   nameOrig        object
 4   oldbalanceOrg   object
 5   newbalanceOrig  object
 6   nameDest        object
 7   oldbalanceDest  object
 8   newbalanceDest  object
 9   isFraud         int64 
 10  isFlaggedFraud  int64 
dtypes: int64(3), object(8)
memory usage: 253.6+ MB


In [ ]:
data.isna().sum()

In [ ]:
data.isnull().sum()

## Data Cleaning 

In [15]:
# Although the project is not actually in USD, it will be easier to understand the context and understand outliers 
# better by viewing the necessary columns as currency 

data['amount'] = data['amount'].apply(lambda x: f"${x:,.2f}")
data['oldbalanceOrg'] = data['oldbalanceOrg'].apply(lambda x: f"${x:,.2f}")
data['newbalanceOrig'] = data['newbalanceOrig'].apply(lambda x: f"${x:,.2f}")
data['oldbalanceDest'] = data['oldbalanceDest'].apply(lambda x: f"${x:,.2f}")
data['newbalanceDest'] = data['newbalanceDest'].apply(lambda x: f"${x:,.2f}")

ValueError: Unknown format code 'f' for object of type 'str'

In [ ]:
# Confirming changes were made to the dataframe 
data.head(20)

### Determining Relevant Columns and Sub-groups to minimize less relevant data to the model 

A few other columns that seem relevant to the fraud detection model are the different balance columns that show the transaction amounts in refernce to the original balances. This is helpful in fraud detection because a targeted account would most likely show a large depletion in their account relevant to the balance they currently held in the account already. 

Hypothesized highly relevant columns to the fraud detection model: 
**OldBalanceOrg: The origin accounts balance before the transaction** <br> 

**NewBalanceOrg: The origin accounts balance after the transaction**<br>

The last columns that are most relevant to the model is the destination in which the money is going. There are three columnns that deal with destination of the transaction amount, however it's unclear what the "nameDest" is referencing if only comparing to the original account number. However, if any "Destination"- account appears amongst multiple different "Origins": ['NameOrig'], then it may be an account that commits frequent fraud. The new balance and old balance destination appear to be a good predictor of potential fraud as well. 

In [9]:
data['type'].unique()


array(['TRANSFER', 'CASH_OUT'], dtype=object)

### Key Indicators of Fraud <br>

**Transaction Type:**
- Fraud is mostly found in "TRANSFER" and "CASH_OUT" transactions involving moving money between accounts.
- Transactions like "DEBIT", "PAYMENT", and "CASH_IN" are most likely not malignant.<br>

**Balance Changes:**
- The sender's (NameOrig) OldBalanceOrg is higher, but the NewBalanceOrg is suspiciously 0, meaning they transferred all their funds out
- The receiver's (NameDest) OldBalanceDest is 0 or unusually low, meaning it could be a newly created fraudulent account <br>

**Large Amounts:**
- Fraudulent transactions often involve large amounts being transferred quickly (step)
- The IsFlaggedFraud column may flag a transaction if it exceeds 200,000, but actual fraud can occur below this threshold- it's not clear what currency this is solely from the data so context is important <br>

**Destination Account Behavior:**
- If the destination account (NameDest) is frequently receiving large sums and quickly cashing out, it might be part of a fraud network
- May see the same accounts used multiple times or completely new accounts ($0 balance) to launder money <br>

**Repeated Transactions in a Short Time:**
- If multiple transactions occur within a short time from the same NameOrig, it could indicate an account being drained <br>

In [7]:
# Separate numerical and categorical features
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

# Check the results
print(f"Numerical Features: {numerical_features}")
print(f"Categorical Features: {categorical_features}")


Numerical Features: Index(['step', 'isFraud', 'isFlaggedFraud'], dtype='object')
Categorical Features: Index(['type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest'],
      dtype='object')


### Initial Data Visualizations to Determine Most Relevant Features 

In [ ]:
# Bivariate Analysis - Amount vs Fraud (Boxplot using Log scale)
sns.boxplot(x='isFraud', y='amount', data=data)
plt.yscale('log')  
plt.show()


*Class Imbalance:*
It was a bit difficult to truly view the data through box plots with such a large amount of data. This boxplot uses logarithmic scale to view the large amounts of data more clearly. This way, the data is "stretched" and easier to make comparisons. It may be even clearer to use a sample of the data instead as well. 

The boxplot shows a significant number of outliers in the 'Not Fraud' (0) category, which is expected as large legitimate transactions naturally occur. Fraudulent transactions may follow different patterns, which could be explored further

In [ ]:

# Univariate Analysis - Distribution of 'type' column
plt.figure(figsize=(8, 6))
sns.countplot(data['type'])
plt.title('Distribution of Transaction Types')
plt.xlabel('Type')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()




In the graph above, the distribution of transaction types shows the different types of transactions made at each instance. Since this is a fraud detection model, it might not be necessary to take into account a transaction type such as "CASH_IN". A transaction type of adding money into an account very rarely would be fraudulent and even if it were, the transactions that will be most likely to be high predictors of fraud would be high values in 'CASH_OUT' or 'TRANSFER' 

In [ ]:
# Bivariate Analysis - Transaction Type vs Fraud
plt.figure(figsize=(8, 6))
sns.countplot(x='type', hue='isFraud', data=data)
plt.title('Transaction Type vs Fraud')
plt.xlabel('Transaction Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()



Although difficult to view due to the large class imbalance between Not Fraud '0' and Fraud '1', it is evident in the above graph that the current markers for Fraud (shown in orange) are generalized in the two transaction types: *TRANSFER* and *CASH_OUT*


#### Potential Columns to Drop: <br>
- [`nameOrig`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A194%2C%22character%22%3A40%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A432%2C%22character%22%3A48%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A922%2C%22character%22%3A44%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A117%2C%22character%22%3A40%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") - Not as useful as [`nameDest`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A124%2C%22character%22%3A14%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A944%2C%22character%22%3A15%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A444%2C%22character%22%3A36%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") for identifying transaction patterns. <br>
- Rows with [`DEBIT`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X54sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A5%2C%22character%22%3A2%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A178%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A766%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1181%2C%22character%22%3A7%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition"), [`CASH_IN`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X54sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A5%2C%22character%22%3A9%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1181%2C%22character%22%3A14%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition"), and [`PAYMENT`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X54sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A5%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A101%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A108%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A640%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1181%2C%22character%22%3A23%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") values in the [`type`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X54sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A5%2C%22character%22%3A41%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A117%2C%22character%22%3A21%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A194%2C%22character%22%3A21%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A432%2C%22character%22%3A27%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A922%2C%22character%22%3A22%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1150%2C%22character%22%3A13%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") column - These transaction types are not relevant for fraud detection as fraud is primarily associated with [`TRANSFER`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A73%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A80%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A433%2C%22character%22%3A23%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A668%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A318%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1169%2C%22character%22%3A235%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") and [`CASH_OUT`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A87%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A434%2C%22character%22%3A23%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A94%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A332%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A682%2C%22character%22%3A18%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1169%2C%22character%22%3A250%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition"). <br>
- [`isFlaggedFraud`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A124%2C%22character%22%3A65%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A456%2C%22character%22%3A17%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A944%2C%22character%22%3A64%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") - This column has limited utility as it only flags transactions above a specific threshold, which may not be comprehensive. <br>

#### Proposed New Columns: <br>
- [`step`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22vscode-notebook-cell%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22X54sZmlsZQ%3D%3D%22%7D%2C%22pos%22%3A%7B%22line%22%3A8%2C%22character%22%3A2%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A117%2C%22character%22%3A11%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A194%2C%22character%22%3A11%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A432%2C%22character%22%3A17%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A922%2C%22character%22%3A12%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A1184%2C%22character%22%3A7%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") - Could be useful in a new context, such as identifying time-based patterns in fraudulent activity. <br>
- `transactionDifference` - The difference between [`oldbalanceOrg`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A117%2C%22character%22%3A50%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A432%2C%22character%22%3A58%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A922%2C%22character%22%3A53%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A194%2C%22character%22%3A50%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") and [`newbalanceOrig`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A117%2C%22character%22%3A65%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A194%2C%22character%22%3A65%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A444%2C%22character%22%3A17%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A922%2C%22character%22%3A67%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") to identify anomalies in transaction amounts. <br>
- `balanceDestChange` - The difference between [`oldbalanceDest`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A124%2C%22character%22%3A24%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A944%2C%22character%22%3A24%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A444%2C%22character%22%3A46%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") and [`newbalanceDest`](command:_github.copilot.openSymbolFromReferences?%5B%22%22%2C%5B%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A124%2C%22character%22%3A40%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A444%2C%22character%22%3A62%7D%7D%2C%7B%22uri%22%3A%7B%22scheme%22%3A%22file%22%2C%22authority%22%3A%22%22%2C%22path%22%3A%22%2FUsers%2Fsa27%2FTLAB_FD%2FTLAB_Fraud_Detect%2FJupyter%20Notebooks%2Ffraud_detect_eda.ipynb%22%2C%22query%22%3A%22%22%2C%22fragment%22%3A%22%22%7D%2C%22pos%22%3A%7B%22line%22%3A944%2C%22character%22%3A39%7D%7D%5D%2C%22fb307630-26d5-4448-bc9b-5263fd7986c2%22%5D "Go to definition") to track unexpected changes in destination balances. <br>
- `isHighValue` - A binary column indicating whether the transaction amount exceeds a certain threshold, which could help identify high-risk transactions. <br> 

These edits aim to streamline the dataset for fraud detection by focusing on relevant features and removing noise. #Reference: Github Copilot 

In [8]:
# Filter out rows with irrelevant transaction types
data = data[~data['type'].isin(['DEBIT', 'CASH_IN', 'PAYMENT'])]

# New columns
data['transactionDifference'] = data['oldbalanceOrg'] - data['newbalanceOrig']
data['balanceDestChange'] = data['newbalanceDest'] - data['oldbalanceDest']
data['isHighValue'] = (data['amount'] > 100000).astype(int)
data['isAccountDrained'] = (data['newbalanceOrig'] == 0).astype(int)
data['isReceiverNew'] = (data['oldbalanceDest'] == 0).astype(int)
data['transactionFrequency'] = data.groupby('nameOrig')['step'].transform('count')
data['isRepeatedReceiver'] = data.groupby('nameDest')['nameDest'].transform('count') >1
data['isLargeAmountFlagged'] = (data['amount'] >= 190000).astype(int)

# Drop old columns that are no longer needed
columns_to_drop = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
data = data.drop(columns=columns_to_drop)

type(data)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
data.head(20)

## Explanation of New Columns:
1. **`transactionDifference`**: Helps identify if the sender transferred all their funds, which is a common fraud indicator [Replaces OldbalanceOrg & newbalanceOrg]
2. **`balanceDestChange`**: Tracks changes in the receiver's balance to detect unusual behavior [Replaces OldbalanceDest & newbalanceDest]
3. **`isHighValue`**: Flags high-value transactions that may be high-risk.
4. **`isAccountDrained`**: Identifies accounts that are completely drained after a transaction.
5. **`isReceiverNew`**: Flags transactions where the receiver's account is new or has no prior balance.
6. **`transactionFrequency`**: Detects repeated transactions from the same sender in a short time, which could indicate account draining.
7. **`isRepeatedReceiver`**: Flags receivers that are frequently used, which could indicate a fraud network.
8. **`isLargeAmountFlagged`**: Flags transactions close to the fraud threshold (e.g., 200,000), even if they are not explicitly flagged.



In [ ]:
# Summary statistics for new columns
new_columns = ['balanceDestChange', 'isHighValue', 'isAccountDrained', 'isReceiverNew', 
               'transactionFrequency', 'isRepeatedReceiver', 'isLargeAmountFlagged', 'transactionDifference']
data[new_columns].describe()

In [ ]:
# Correlation with isFraud
correlation = data[new_columns + ['isFraud']].corr()
print(correlation['isFraud'].sort_values(ascending=False))

In [ ]:
# Heatmap for correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

isReceiverNew has a high correlation to our current fraud detection model at 0.080177 and highest correlation is between 'transactionDifference' and isFraud at 0.474628 showing a high marker for fraud in our current model. 

The current model shows an inverse relationship with 'isFraud' for 'isRepeatedReceiver' at -0.085970, so it its important o further analyze whether this is an issue with the feature itself or the current model's ability to flag this feature as relevant fraud.


In [ ]:
# Boxplots for fraud vs non-fraud
for col in ['balanceDestChange', 'transactionDifference', 'transactionFrequency']:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x='isFraud', y=col, data=data)
    plt.title(f'{col} vs Fraud')
    plt.yscale('log')  # Use log scale if there are large outliers
    plt.show()

In [ ]:
print(data['transactionFrequency'].value_counts())
print(data['transactionFrequency'].describe())

In [ ]:
# Calculating raud rate for each unique transaction frequency
fraud_rate = data.groupby('transactionFrequency')['isFraud'].mean()
fraud_rate.plot(kind='bar', figsize=(8, 4), title='Fraud Rate by Transaction Frequency')
plt.ylabel('Fraud Rate')
plt.xlabel('Transaction Frequency')
plt.show()

## Model Selection and Evaluation


In [ ]:
# Assigning features (X) and target (y)
X = data.drop(columns=['isFraud'])  # Drop the target column from features
y = data['isFraud']  # Target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display the shapes of the datasets to confirm proper sizes 
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

The **80-20 split** is correctly applied

1. **`X_train shape: (2216327, 15)`**:
   - The training set contains **2,216,327 rows** and **15 features** (columns)
   - These are the independent variables used to train the model

2. **`X_test shape: (554082, 15)`**:
   - The testing set contains **554,082 rows** and **15 features**
   - These are the independent variables used to evaluate the model

3. **`y_train shape: (2216327,)`**:
   - The training set contains **2,216,327 target labels** 

4. **`y_test shape: (554082,)`**:
   - The testing set contains **554,082 target labels** 







In [ ]:
print(X_train.dtypes)

Will need to use one hot encoding for the categorical columns:

In [ ]:
# Removing non-numeric columns and one-hot encoding the 'type' column 
X_train = X_train.drop(columns=['nameOrig', 'nameDest'])
X_test = X_test.drop(columns=['nameOrig', 'nameDest'])

X_train = pd.get_dummies(X_train, columns=['type'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['type'], drop_first=True)

X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
print(X_train.dtypes)
print(X_test.dtypes)


### Model #1 Logistic Regression



In [ ]:
# Initialize & train model
logistic_model = LogisticRegression(random_state=42, max_iter=1000)
logistic_model.fit(X_train, y_train)

# Make predictions
y_pred = logistic_model.predict(X_test)
y_pred_proba = logistic_model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Precision for fraud (0.93): The model is correct 93% of the time when predicting fraud. False positives are relatively low. <br>
Recall for fraud (0.60): The model only catches 60% of actual fraud cases, meaning it's missing 40% of real fraud cases (false negatives) <br>
F1-score (0.73): Since recall is low, the F1-score is not ideal

- 665 fraud cases were missed(false Negatives) which is a major issue <br>
- Only 72 legitimate transactions were wrongly flagged as fraud (False Positives), which is pretty low
- Severe class imbalance (552,439 vs. 1,643 fraud cases) is causing the model to favor predicting 'Not Fraud' more often



### Model #2 SVM 


In [ ]:
# Initialize & train the modelSVM model
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train, y_train)

# Predictions
y_pred = svm_model.predict(X_test)
y_pred_proba = svm_model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class (isFraud=1)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Calculate and plot the ROC-AUC curve
roc_auc = roc_auc_score(y_test, y_pred_proba)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"SVM (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


### Model #3: Random Forest

##  Downsampling and SMOTE
#### Taking a smaller sample of the whole dataset to get a better picture of the data with limited outliers and dealing with class imbalance

In [ ]:
# Sample a portion of the data

#print(data[pairplot_columns].head(50))
data_sampled = data.sample(frac=0.1, random_state=100)

print(data['isFraud'])
print(type(data['isFraud']))


In [ ]:
# Here I was having difficulties because the sample I created wasn't showing up as a pandas dataframe with each column as a series. Instead, the columns were also dataframes which I have  resolved
# 
print(type(data_sampled['isFraud']))  


In [ ]:
print(data_sampled['isFraud'].dtype)   


In [ ]:
data['isFraud'].value_counts()


In [ ]:
data['isFraud'].value_counts(normalize=True) * 100


In the two cells above, it's a bit more clear to view the imbalance between the two classes. This makes sense, as we can imagine that the baseline of fraud detection would be that a transaction is flagged as "Not Fraud", which is classified as 0.  

In order to deal with this issue, we can use Synthetic Minority Over-sampling Technique (SMOTE) to address the issue of the minority class. 


## 